In [ ]:
# Scispacy scibert model

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, convert_to_messages
from os.path import dirname, join
from dotenv import load_dotenv
from typing import Literal
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, MessagesState, END
from langgraph.types import Command
from operator import add
from Agent import BTEx
import pandas as pd
import getpass
import os
import json

In [ ]:
# Set your OpenAI API key
load_dotenv("/Users/mastorga/Documents/BTE-LLM/.env")

if not os.environ.get("OPENAI_API_KEY"): #field to ask for OpenAI API key
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter OpenAI API Key: ")

In [ ]:
# Create an LLM-based agent
llm = ChatOpenAI(temperature=0, model="gpt-4o")  # Change model if needed

In [ ]:
def get_llm_response(question):
    print(question + '\n\n')
    
    response = llm.invoke(question).content

    print(f"LLM answer: {response}")
    
    return response

In [ ]:
f = open("/Users/mastorga/Documents/BTE-LLM/Prototype/data/MCQ_2hop.json")

MCQ2hop = json.load(f)

print(len(MCQ2hop))

In [ ]:
MCQ2hop_test = [{'question': 'Which of the following cellular components is linked to the genes SDHC and SDHA ? 1. caveola 2. integral component of lumenal side of endoplasmic reticulum membrane 3. intermediate-density lipoprotein particle 4. fumarate reductase complex 5. GARP complex', 'answer': '4'}, {'question': 'Which of the following is a gene which interacts with the genes GDI2 and SEC23A ? 1. LOC127401542 2. GLRA2 3. LOC127271640 4. SYT6 5. MPTX1', 'answer': '2'}, {'question': 'Which of the following biological processes does not involve both the genes ALDOA and HIF1A ? 1. mismatch repair 2. regulation of body fluid levels 3. regulation of anatomical structure morphogenesis 4. cellular homeostasis 5. wound healing', 'answer': '1'}, {'question': 'Which of the following is not a gene commonly involved in the pathways TNFalpha and RIG-I-like receptor signaling pathway - Homo sapiens (human) ? 1. MAPK8 2. TRAF6 3. CHUK 4. MAPK14 5. LOC127401194', 'answer': '5'}, {'question': 'Which of the following increases the gene expression of CASP10 and KLHL9 ? 1. Oprozomib 2. ANZ-100 3. Olopatadine 4. IPL-344 5. Sarizotan', 'answer': '3'}, {'question': 'Which of the following is not a gene commonly associated with the cellular components transferase complex, transferring phosphorus-containing groups and ribose phosphate diphosphokinase complex ? 1. PRPSAP1 2. PRPS2 3. LINC02731 4. PRPSAP2 5. PRPS1', 'answer': '3'}, {'question': 'Which of the following molecular functions is not attributed to both the genes PLAAT3 and ABHD3 ? 1. phospholipase A2 activity 2. carboxylic ester hydrolase activity 3. phosphatidylserine 1-acylhydrolase activity 4. 3-chloroallyl aldehyde dehydrogenase activity 5. phosphatidylcholine 1-acylhydrolase activity', 'answer': '4'}, {'question': "Which of the following can be treated by the drugs Haloperidol and Benzatropine ? 1. Alzheimer Disease 14 2. Alzheimer Disease, Early Onset 3. Familial Alzheimer Disease (FAD) 4. Alzheimer's Disease 5. Alzheimer Disease, Late Onset", 'answer': '4'}, {'question': 'Which of the following pathways contains the genes ABCG2 and ABCB1 ? 1. Activated NTRK3 signals through PI3K 2. Lamivudine Pathway, Pharmacokinetics/Pharmacodynamics 3. Glycosphingolipid metabolism 4. Defective ABCC2 causes Dubin-Johnson syndrome 5. Betazole Action Pathway', 'answer': '2'}, {'question': 'Which of the following binds to the genes SLC22A6 and SLC22A11 ? 1. Amylose 2. Selurampanel 3. 2-(Beta-D-Glucopyranosyl)-5-Methyl-1,2,3-Benzimidazole 4. Diclofenac 5. Reparixin', 'answer': '4'}, {'question': 'Which of the following is not a gene subject to increased expression by both the drugs Dexamethasone and Betamethasone ? 1. KLF2 2. GADD45B 3. GSPT1 4. LOC124904253 5. EIF4A3', 'answer': '4'}, {'question': "Which of the following genes is not commonly associated with the diseases Alzheimer's Disease and Alzheimer Disease, Early Onset ? 1. CRH 2. HLA-DRB5 3. MIR100 4. LOC127406786 5. GAPDHS", 'answer': '4'}, {'question': "Which of the following is not a symptom of both Alzheimer's Disease and Familial Alzheimer Disease (FAD) ? 1. Mydriasis 2. Memory Disorders 3. Primary Progressive Nonfluent Aphasia 4. Myoclonus 5. Agnosia", 'answer': '1'}, {'question': 'Which of the following is a gene associated with the cellular components cell-substrate adherens junction and cytoplasmic region ? 1. MICB 2. LOC129389012 3. LOC105370573 4. ZNF299P 5. TRPV4', 'answer': '5'}, {'question': 'Which of the following is not a gene which commonly performs the molecular functions hydrolase activity, acting on acid anhydrides and nucleoside-triphosphatase activity ? 1. GNL3 2. LOC127888933 3. ATP11B 4. HSPA6 5. TUBB8', 'answer': '2'}, {'question': 'Which of the following drugs does not commonly bind to the genes PTGS1 and ALOX5 ? 1. Mesalazine 2. Olsalazine 3. Lometrexol 4. Meclofenamic acid 5. Diethylcarbamazine', 'answer': '3'}, {'question': 'Which of the following biological processes does not involve both the genes GNAT1 and EGLN1 ? 1. negative regulation of catalytic activity 2. organ morphogenesis 3. cell-substrate adhesion 4. regulation of cyclic-nucleotide phosphodiesterase activity 5. negative regulation of hydrolase activity', 'answer': '3'}, {'question': 'Which of the following is a gene in the pathways Herpes simplex infection - Homo sapiens (human) and Generic Transcription Pathway ? 1. LOC127882693 2. LOC127406241 3. ZNF705A 4. PHOX2A 5. LOC127398906', 'answer': '3'}, {'question': 'Which of the following is a gene in the pathways Nuclear Receptors Meta-Pathway and NRF2 pathway ? 1. LOC127889773 2. SLC39A13 3. ESAM-AS1 4. LOC129664639 5. SNORA2C', 'answer': '2'}, {'question': 'Which of the following is not a gene which commonly participates in the biological processes maturation of LSU-rRNA and rRNA processing ? 1. LOC102725000 2. RPF1 3. BOP1 4. NOL9 5. RPL7L1', 'answer': '1'}, {'question': 'Which of the following is a gene associated with the diseases Alzheimer Disease, Early Onset and Alzheimer Disease, Late Onset ? 1. LOC127458948 2. LOC122787139 3. TF 4. LOC127403702 5. LOC127828867', 'answer': '3'}, {'question': 'Which of the following genes is not over-expressed in both prostate gland and testis ? 1. TMED3 2. PDE4DIPP2 3. SOX4 4. WDR90 5. TPSD1', 'answer': '2'}, {'question': 'Which of the following is a gene in the pathways Ferroptosis and Amino acid and oligopeptide SLC transporters ? 1. DEL10Q26 2. MGAT4EP 3. NNO3 4. SLC7A11 5. LOC127897268', 'answer': '4'}, {'question': 'Which of the following increases the gene expression of CNOT4 and BCL7B ? 1. 3-(4-{2-[2-(2-bromo-acetylamino)-ethyldisulfanyl]-ethylcarbamoyl}-cyclohexylcarbamoyl)-pyrazine-2-carboxylic acid 2. Cyclohexyl-Hexyl-Beta-D-Maltoside 3. Indigotindisulfonic acid 4. Reboxetine 5. Triheptanoin', 'answer': '4'}, {'question': 'Which of the following is not a drug which commonly decreases expression of the genes CDCA4 and PIN1 ? 1. 9-(6-deoxy-beta-D-allofuranosyl)-6-methylpurine 2. Cyclosporine 3. Terbinafine 4. Pitavastatin 5. Thioridazine', 'answer': '1'}, {'question': 'Which of the following is a gene which participates in the biological processes regulation of chemotaxis and inflammatory response ? 1. LOC124902573 2. LOC127897982 3. CXCL8 4. MIR6877 5. LOC127817259', 'answer': '3'}, {'question': 'Which of the following is not a gene which commonly interacts with the genes IQCB1 and UBC ? 1. REXO1L2P 2. RAB14 3. TUBA4A 4. CALM2 5. FYCO1', 'answer': '1'}, {'question': 'Which of the following is a gene in the pathways EGFR1 and MAPK signaling pathway - Homo sapiens (human) ? 1. RPL36AP47 2. STUT4 3. MTND2P25 4. LOC127897017 5. PTPRR', 'answer': '5'}, {'question': 'Which of the following is not a gene which commonly performs the molecular functions kinase activity and phosphotransferase activity, alcohol group as acceptor ? 1. GSK3B 2. MST1R 3. LOC127883612 4. KALRN 5. STK32C', 'answer': '3'}, {'question': "Which of the following is associated with the genes CRH and PLCG2 ? 1. Familial Alzheimer-like prion disease 2. Non-Alzheimer's dementia (e.g., Lewy body dementia, vascular or multi-infarct dementia; mixed dementia; frontotemporal dementia such as Pick's disease; and dementia related to stroke, Parkinson's or Creutzfeldt-Jakob diseases) 3. Familial Alzheimer Disease (FAD) 4. Alzheimer Disease 12 5. Alzheimer Disease, Familial, 3, with Spastic Paraparesis and Unusual Plaques", 'answer': '3'}, {'question': 'Which of the following genes performs the molecular functions oxidoreductase activity, oxidizing metal ions and oxidoreductase activity ? 1. FTH1 2. LOC127895777 3. HECTD3 4. LOC127893025 5. LOC127273811', 'answer': '1'}, {'question': "Which of the following decreases the gene expression of NPDC1 and TNIP1 ? 1. Aspartic Acid-4-Carboxymethyl Ester 2. 2-[(2',3',4'-TRIFLUOROBIPHENYL-2-YL)OXY]ETHANOL 3. Baloxavir 4. Diclofenac 5. 1,4-dithio-alpha-D-glucopyranose", 'answer': '4'}, {'question': 'Which of the following is a gene in the pathways Oxidative Stress Induced Senescence and miR-targeted genes in epithelium - TarBase ? 1. LOC127892904 2. CDKN2A 3. LOC127268090 4. LOC127275845 5. LOC126861236', 'answer': '2'}, {'question': 'Which of the following drugs does not commonly bind to the genes CYP2D6 and CYP2A6 ? 1. Rosiglitazone 2. Progesterone 3. Human papillomavirus type 31 L1 capsid protein antigen 4. Dapagliflozin 5. Arformoterol', 'answer': '3'}, {'question': 'Which of the following classes includes the drugs Theophylline and Aminophylline ? 1. Prostaglandins I 2. Xanthines 3. Decreased Immunologically Active Molecule Activity 4. Prostaglandins 5. Oxazolidinones', 'answer': '2'}, {'question': 'Which of the following is not a gene subject to increased expression by both the drugs Dasatinib and Bortezomib ? 1. PIK3C3 2. PHGDH 3. ABHD4 4. CEP170 5. RGS2', 'answer': '4'}, {'question': 'Which of the following belongs to the drug classes Calcium Channel Antagonists and Dihydropyridines ? 1. Rheinanthrone 2. Trimazosin 3. Clevidipine 4. Chloroform 5. 4-HYDROXY-5-IODO-3-NITROPHENYLACETYL-EPSILON-AMINOCAPROIC ACID ANION', 'answer': '3'}, {'question': "Which of the following can be affected by the drugs Ziprasidone and Naltrexone ? 1. Alzheimer's Disease 2. Alzheimer disease type 1 3. Alzheimer Disease 7 4. Alzheimer Disease, Familial, 3, with Spastic Paraparesis and Unusual Plaques 5. Lewy Body Variant of Alzheimer Disease", 'answer': '1'}, {'question': 'Which of the following increases the gene expression of SESN1 and CNOT4 ? 1. Liraglutide 2. Verapamil 3. Zenocutuzumab 4. Methionine Phosphinate 5. (2S)-2-(4-ethylphenoxy)-3-phenylpropanoic acid', 'answer': '2'}, {'question': "Which of the following is a symptom of Alzheimer's Disease and Familial Alzheimer Disease (FAD) ? 1. Hypercalciuria 2. Gait Ataxia 3. Emaciation 4. Gastroparesis 5. Apraxias", 'answer': '5'}, {'question': 'Which of the following is not a gene subject to increased expression by both the drugs Betamethasone and Pitavastatin ? 1. TSC22D3 2. LOC127268675 3. PLCB3 4. PLA2G15 5. MVP', 'answer': '2'}, {'question': 'Which of the following is not a gene which binds to both the drugs Sunitinib and Bosutinib ? 1. RN7SL557P 2. ABCB1 3. PHKG1 4. HIPK4 5. NUAK2', 'answer': '1'}, {'question': 'Which of the following genes is over-expressed in trachea and brain ? 1. AKAP14 2. LOC127274682 3. LOC127457306 4. LOC107985827 5. LOC127820719', 'answer': '1'}, {'question': 'Which of the following genes is not over-expressed in both lung and heart ? 1. SNRPG 2. FAM107A 3. SLC12A4 4. ID3 5. LOC127398478', 'answer': '5'}, {'question': 'Which of the following molecular functions is not attributed to both the genes MRM1 and FTSJ3 ? 1. O-methyltransferase activity 2. S-adenosylmethionine-dependent methyltransferase activity 3. calcium ion binding 4. transferase activity, transferring one-carbon groups 5. RNA methyltransferase activity', 'answer': '3'}, {'question': 'Which of the following is a gene which participates in the biological processes regulation of gene expression, epigenetic and regulation of protein serine/threonine kinase activity ? 1. DIRAS3 2. LOC127399861 3. LOC129660990 4. LOC127819192 5. LOC127274335', 'answer': '1'}, {'question': 'Which of the following genes is over-expressed in testis and trachea ? 1. LOC127824021 2. RPL23AP12 3. LOC127458103 4. AACS 5. H2BC20P', 'answer': '4'}, {'question': 'Which of the following binds to the drugs Methadone and Imipramine ? 1. LOC127830012 2. CYP2C18 3. AKR7L 4. LOC124908462 5. RNU6-862P', 'answer': '2'}, {'question': 'Which of the following genes is over-expressed in cardiac atrium and trigeminal ganglion ? 1. LOC127884932 2. PDLIM7 3. LOC124902905 4. LOC127822321 5. KCTD9', 'answer': '5'}, {'question': 'Which of the following increases the gene expression of TCTA and YME1L1 ? 1. Niraparib 2. Mucor circinelloides f. lusitanicus 3. N-[2-(5-methyl-4H-1,2,4-triazol-3-yl)phenyl]-7H-pyrrolo[2,3-d]pyrimidin-4-amine 4. Chlorcyclizine 5. SB-743921', 'answer': '4'}]

print(MCQ2hop_test)


In [ ]:
# LLM only
LLM_tally = {}
LLM_correctct = 0

for index, item in enumerate(MCQ2hop_test):
    print(f"Question {index+1}. \n")
    question = "IMPORTANT: only return the number corresponding to the correct answer. \n" + item.get("question", "")

    LLM_ans = get_llm_response(question)

    groundtruth = item.get("answer", "")

    print(f"Correct answer: {groundtruth}\n\n")

    if groundtruth in LLM_ans:
        LLM_tally[index+1] = "CORRECT"
        print("\nCORRECT")
        LLM_correctct += 1
    else:
        LLM_tally[index+1] = "INCORRECT"
        print("\nINCORRECT")

    print("----------------------------------")

print(LLM_tally)
print(f"\n Score: {LLM_correctct}/{len(MCQ2hop_test)} \n Accuracy: {LLM_correctct/len(MCQ2hop_test)}")

In [ ]:
# BTE-RAG 50m5k
BTEx_50m5k_tally = {}
BTEx_50m5k_correctct = 0

for index, item in enumerate(MCQ2hop_test):
    print(f"Question {index+1}. \n")
    question = "IMPORTANT: only return the number corresponding to the correct answer. \n" + item.get("question", "")

    BTEx_50m5k_ans = BTEx(question, maxresults=50, k=5)

    groundtruth = item.get("answer", "")

    print(f"Correct answer: {groundtruth}\n\n")

    if BTEx_50m5k_ans:
        if groundtruth in BTEx_50m5k_ans:
            BTEx_50m5k_tally[index+1] = "CORRECT"
            print("\nCORRECT")
            BTEx_50m5k_correctct += 1
        else:
            BTEx_50m5k_tally[index+1] = "INCORRECT"
            print("\nINCORRECT")
    else:
        BTEx_50m5k_tally[index+1] = "NO ANSWER"
        print("\nNO ANSWER")

    print("----------------------------------")

print(BTEx_50m5k_tally)
print(f"\n Score: {BTEx_50m5k_correctct}/{len(MCQ2hop_test)} \n Accuracy: {BTEx_50m5k_correctct/len(MCQ2hop_test)}")

In [ ]:
# BTE-RAG 50m10k
BTEx_50m10k_tally = {}
BTEx_50m10k_correctct = 0

for index, item in enumerate(MCQ2hop_test):
    print(f"Question {index+1}. \n")
    question = "IMPORTANT: only return the number corresponding to the correct answer. \n" + item.get("question", "")

    BTEx_50m10k_ans = BTEx(question, maxresults=50, k=10)

    groundtruth = item.get("answer", "")

    print(f"Correct answer: {groundtruth}\n\n")

    if BTEx_50m10k_ans:
        if groundtruth in BTEx_50m10k_ans:
            BTEx_50m10k_tally[index+1] = "CORRECT"
            print("\nCORRECT")
            BTEx_50m10k_correctct += 1
        else:
            BTEx_50m10k_tally[index+1] = "INCORRECT"
            print("\nINCORRECT")
    else:
        BTEx_50m10k_tally[index+1] = "NO ANSWER"
        print("\nNO ANSWER")

    print("----------------------------------")

print(BTEx_50m10k_tally)
print(f"\n Score: {BTEx_50m10k_correctct}/{len(MCQ2hop_test)} \n Accuracy: {BTEx_50m10k_correctct/len(MCQ2hop_test)}")

In [ ]:
# BTE-RAG 100m5k
BTEx_100m5k_tally = {}
BTEx_100m5k_correctct = 0

for index, item in enumerate(MCQ2hop_test):
    print(f"Question {index+1}. \n")
    question = "IMPORTANT: only return the number corresponding to the correct answer. \n" + item.get("question", "")

    BTEx_100m5k_ans = BTEx(question, maxresults=100, k=5)

    groundtruth = item.get("answer", "")

    print(f"Correct answer: {groundtruth}\n\n")

    if BTEx_100m5k_ans:
        if groundtruth in BTEx_100m5k_ans:
            BTEx_100m5k_tally[index+1] = "CORRECT"
            print("\nCORRECT")
            BTEx_100m5k_correctct += 1
        else:
            BTEx_100m5k_tally[index+1] = "INCORRECT"
            print("\nINCORRECT")
    else:
        BTEx_100m5k_tally[index+1] = "NO ANSWER"
        print("\nNO ANSWER")

    print("----------------------------------")

print(BTEx_100m5k_tally)
print(f"\n Score: {BTEx_100m5k_correctct}/{len(MCQ2hop_test)} \n Accuracy: {BTEx_100m5k_correctct/len(MCQ2hop_test)}")

In [ ]:
# BTE-RAG 100m10k
BTEx_100m10k_tally = {}
BTEx_100m10k_correctct = 0

for index, item in enumerate(MCQ2hop_test):
    print(f"Question {index+1}. \n")
    question = "IMPORTANT: only return the number corresponding to the correct answer. \n" + item.get("question", "")

    BTEx_100m10k_ans = BTEx(question, maxresults=100, k=10)

    groundtruth = item.get("answer", "")

    print(f"Correct answer: {groundtruth}\n\n")

    if BTEx_100m10k_ans:
        if groundtruth in BTEx_100m10k_ans:
            BTEx_100m10k_tally[index+1] = "CORRECT"
            print("\nCORRECT")
            BTEx_100m10k_correctct += 1
        else:
            BTEx_100m10k_tally[index+1] = "INCORRECT"
            print("\nINCORRECT")
    else:
        BTEx_100m10k_tally[index+1] = "NO ANSWER"
        print("\nNO ANSWER")

    print("----------------------------------")

print(BTEx_100m10k_tally)
print(f"\n Score: {BTEx_100m10k_correctct}/{len(MCQ2hop_test)} \n Accuracy: {BTEx_100m10k_correctct/len(MCQ2hop_test)}")